# Tutorial

In [1]:
import ccp

Q_ = ccp.Q_

`ccp` uses pint to handle units. Q_ is a pint quantity.

Here lets define a suction pressure `ps` and a suction temperature `Ts`.

For the suction pressure we are going to use pint.

In [2]:
ps = Q_(3, "bar")
Ts = 300

The pint objects hold the magnitude and units for a variable and can be used for unit conversion:

In [3]:
print(
    f"ps: {ps}\n"
    f"ps magnitude: {ps.magnitude}\n"
    f"ps units: {ps.units}\n"
    f"convert to atm: {ps.to('atm')}\n"
)

ps: 3 bar
ps magnitude: 3
ps units: bar
convert to atm: 2.960769800148039 standard_atmosphere



We define the fluid composition is defined as a dictionary:

In [4]:
fluid = {
    "CarbonDioxide": 0.79585,
    "R134a": 0.16751,
    "Nitrogen": 0.02903,
    "Oxygen": 0.00761,
}

We can create the suction and discharge states using the `State` class:

In [5]:
suc0 = ccp.State.define(fluid=fluid, p=ps, T=Ts)
disch0 = ccp.State.define(fluid=fluid, p=Q_(7.255, 'bar'), T=391.1)

Notice that in the cell above we defined the states using pint quantities mixed with pure floats.

The way that `ccp` works is by assuming that float are in the SI units system, so 

```python
disch0 = ccp.State.define(fluid=fluid, p=Q_(7.255, 'bar'), T=391.1)
```

is the same as 

```python
disch0 = ccp.State.define(fluid=fluid, p=Q_(7.255, 'bar'), T=Q_(391.1, 'degK'))
```

Create performance point(s):

In [6]:
point0 = ccp.Point(
    suc=suc0,
    disch=disch0,
    speed=Q_(7941, "RPM"),
    flow_m=Q_(34203.6, "kg/hr"),
    b=0.0285,
    D=0.365,
)
point1= ccp.Point(
    suc=suc0,
    disch=disch0,
    speed=Q_(7941, "RPM"),
    flow_m=Q_(37203.6, "kg/hr"),
    b=0.0285,
    D=0.365,
)

Now we can create an impeller, which is basically a container for points.

In [7]:
imp = ccp.Impeller([point0, point1]) 

In [8]:
imp